# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import json
import datetime
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
filepath = './data/song_data'
song_files = get_files(filepath)
print(song_files[0])
print(len(song_files))

/home/pelielo/projects/nd027-project1/data/song_data/A/A/A/TRAAAAW128F429D538.json
71


In [5]:
json_data = []

for song in song_files: # Remove to load all events
    with open(song) as json_file:
        data = json.load(json_file)
        json_data.append(data)
        
song_df = pd.DataFrame.from_dict(json_data)
song_df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARKFYS91187B98E58F,NaN,NaN,,Jeff And Sheri Easter,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),267.70240,0
2,1,ARNTLGG11E2835DDB9,NaN,NaN,,Clp,SOUDSGM12AC9618304,Insatiable (Instrumental Version),266.39628,0
3,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0
4,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
...,...,...,...,...,...,...,...,...,...,...
66,1,AREVWGE1187B9B890A,-13.44200,-41.99520,Noci (BA),Bitter End,SOFCHDR12AB01866EF,Living Hell,282.43546,0
67,1,AR558FS1187FB45658,NaN,NaN,,40 Grit,SOGDBUF12A8C140FAA,Intro,75.67628,2003
68,1,ARNF6401187FB57032,40.79086,-73.96644,"New York, NY [Manhattan]",Sophie B. Hawkins,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,305.16200,1994
69,1,ARJIE2Y1187B994AB7,NaN,NaN,,Line Renaud,SOUPIRU12A6D4FA1E1,Der Kleine Dompfaff,152.92036,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [6]:
song_data = song_df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [7]:
for song in song_data:
    cur.execute(song_table_insert, tuple(song))
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [8]:
artist_df = song_df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = artist_df.values.tolist()

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [9]:
for artist in artist_data:
    cur.execute(artist_table_insert, tuple(artist))
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [10]:
filepath = './data/log_data'
log_files = get_files(filepath)
print(log_files[0])
print(len(log_files))

/home/pelielo/projects/nd027-project1/data/log_data/2018/11/2018-11-13-events.json
30


In [11]:
json_data = []

for log in log_files: # Remove to load all events
    with open(log) as json_file:
        for line in json_file:
            data = json.loads(line)
            json_data.append(data)
            
# df = pd.read_json(filepath, lines=True) # ALTERNATIVE 
log_df = pd.DataFrame.from_dict(json_data)
log_df = log_df[log_df['page'] == 'NextSong']

print(len(log_df))
# df = df.head(1) # Remove to load all events


log_df.head()

6820


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,1542071549796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,1542079142796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
5,Quad City DJ's,Logged In,Chloe,F,0,Cuevas,451.44771,free,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,506,C'mon N' Ride It (The Train) (LP Version),200,1542081112796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8,Hoobastank,Logged In,Noah,M,1,Chavez,232.17587,free,"Ogden-Clearfield, UT",PUT,NextSong,1.540971e+12,492,Born To Lead,200,1542085206796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,94


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [12]:
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_df = pd.DataFrame()

for i, row in log_df.iterrows():
    timestamp = pd.Series(pd.to_datetime(row['ts'], unit='ms'))
    time_data = (timestamp.values[0], 
             timestamp.dt.hour.values[0], 
             timestamp.dt.day.values[0], 
             timestamp.dt.week.values[0], 
             timestamp.dt.month.values[0], 
             timestamp.dt.year.values[0],
             timestamp.dt.weekday.values[0])
    
    time_df = time_df.append(pd.DataFrame(data = [time_data], columns = column_labels), ignore_index = True)
          
time_df.head()


,start_time,hour,day,week,month,year,weekday
0,2018-11-13 00:40:37.796,0,13,46,11,2018,1
1,2018-11-13 01:12:29.796,1,13,46,11,2018,1
2,2018-11-13 03:19:02.796,3,13,46,11,2018,1
3,2018-11-13 03:51:52.796,3,13,46,11,2018,1
4,2018-11-13 05:00:06.796,5,13,46,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [13]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [14]:
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
1,66,Kevin,Arellano,M,free
3,51,Maia,Burke,F,free
4,9,Wyatt,Scott,M,free
5,49,Chloe,Cuevas,F,free
8,94,Noah,Chavez,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [15]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [16]:
log_df.groupby('song').song.nunique()

song
 I Will Not Reap Destruction    1
#40                             1
'Round Midnight                 1
'Till I Collapse                1
(Hon Vill Ha) Puls              1
                               ..
s.Ada.Licht                     1
shimmer                         1
the king of wishful thinking    1
ÃÂ Noite                      1
ÃÂ Aqui Que Se Passa Tudo     1
Name: song, Length: 5189, dtype: int64

In [17]:
column_labels = ('user_id', 'song_id', 'artist_id', 'start_time', 'session_id', 'level', 'location', 'user_agent')
songplay_df = pd.DataFrame()

for index, row in log_df.iterrows():
    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    song_id, artist_id = results if results else None, None

    # insert songplay record
    songplay_data = ( # songplay_id, user_id, song_id, artist_id, start_time, session_id, level, location, user_agent
        row.userId, 
        song_id, 
        artist_id, 
        pd.Timestamp(row.ts, unit='ms'),
        row.sessionId,
        row.level,
        row.location,
        row.userAgent)
    
    songplay_df = songplay_df.append(pd.DataFrame(data = [songplay_data], columns = column_labels), ignore_index = True)

In [18]:
for i, row in songplay_df.iterrows():
    cur.execute(songplay_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [19]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.